In [ ]:
%matplotlib inline

import matplotlib.pyplot as plt
import numpy as np
import torch as pt
import time
import sys

sys.path.insert(0, '..')

from NNsolver import NNSolver
from NNarchitectures import DenseNet_g
from problems import AllenCahn
from utilities import get_X_process, plot_NN_evaluation, compute_PDE_loss

%load_ext autoreload
%autoreload 2

device = pt.device('cuda')

In [ ]:
T = 0.3
problem = AllenCahn(d=100, T=T)
K = 8000
K_batch = 8000
print_every = 500
delta_t = 0.01
sq_delta_t = pt.sqrt(pt.tensor(delta_t))
N = int(np.ceil(T / delta_t))
gradient_steps = 5 * [10000] + (N - 6) * [6000] + [15000]
learning_rates = [0.0002] * (N - 1) + [0.001]

model = NNSolver(problem, 'AllenCahn', learning_rates=learning_rates, gradient_steps=gradient_steps, NN_class=DenseNet_g, K=K, 
                 K_batch=K_batch, delta_t=delta_t, print_every=print_every, method='implicit')

model.Y_n = [DenseNet_g(problem.d, 1, lr=learning_rates[n], problem=problem).to(device) for n in range(N)] + [problem.g]

In [ ]:
model.train()

In [ ]:
fig = plot_NN_evaluation(model, n=12, reference_solution=False, Y_0_true=0.052802)

In [ ]:
pde_loss = compute_PDE_loss(problem, delta_t=0.01, K=1000, Y_n=model.Y_n, vfun=None, testOde=None, seed=44, print_every=10)

In [ ]:
np.mean(pde_loss)

## K = 1000

In [ ]:
T = 0.3
problem = AllenCahn(d=100, T=T)
K = 1000
K_batch = 1000
print_every = 500
delta_t = 0.01
sq_delta_t = pt.sqrt(pt.tensor(delta_t))
N = int(np.ceil(T / delta_t))
gradient_steps = 10 * [12000] + (N - 11) * [6000] + [15000]
learning_rates = [0.0005] * (N - 1) + [0.001]

model_2 = NNSolver(problem, 'AllenCahn', learning_rates=learning_rates, gradient_steps=gradient_steps, NN_class=DenseNet_g, K=K, 
                 K_batch=K_batch, delta_t=delta_t, print_every=print_every, method='implicit')

model_2.Y_n = [DenseNet_g(problem.d, 1, lr=learning_rates[n], problem=problem, arch=[110, 110, 50, 50]).to(device) for n in range(N)] + [problem.g]

In [ ]:
model_2.train()

In [ ]:
pde_loss = compute_PDE_loss(problem, delta_t=0.01, K=1000, Y_n=model.Y_n, vfun=None, testOde=None, seed=44, print_every=10)

In [ ]:
np.mean(pde_loss)